# Statistical learning concepts

This notebook covers concepts from Chapter 2 in the [Introduction to Statistical Learning textbook](https://catalog.library.tamu.edu/Record/in00004735720).

There is a myriad of AI and machine learning applications
 - Protein folding
 - Image generation
 - Climate forecasting

These areas are incredibly different, but the way we craft models for each is largely the same.

There are unifying principles for constructing and evaluating __learning systems__ to solve a vast array of problems.
 - Understanding these principles allows you to better understand when, how and why your model works
 - Develop new models and generalize old models, see commonalities across disciplines
 - Guide you towards more informed model choices


These principles are called __statistical learning theory__. SLT gives us a rigorous framework for
1. Defining __learning__ problems and building models
3. Analyzing the performance of machine learning models (theoretically and empirically)


In [ ]:
# dont run this without changing the directory

%cd '/content/drive/MyDrive/STAT335_Fall2024/statlearning/'

: 

## 1. Statistical learning theory

> SLT is the subfield of Statistics / Computer Science tries to understand when, how, and why machine learning algorithms work.

Statistical learning is not the same as machine learning. Machine learning is about building models. Statistical learning is about understanding models.
- Understand the __predictive skill__ of __learning algorithms__

To be an effective data scientist you need to know the basics of learning theory.

### Overview

To get started in ML we need the following basics from SLT

1. How to define a learning problem
2. How to solve learning problems
3. How to evaluate effectiveness

## 1.1 How to define a learning problem

> Machine learning is the art and science of building __models__ that __learn__ from data to accomplish a set of tasks

Lets reformulate this nebulous goal into something a little more concrete.



### Tasks

All learning problems start with a __task__. A task is something that you would like to do
 - Classify images
 - Predict bitcoin price

but, typically, is hard or inconvenient to do.

In this class we will focus almost exclusively on tasks that involve __predicting__ one random variable $Y$ from another random variable $X$.

### Solving tasks

Suppose we have a task that we want to complete (or automate). We're going to teach a machine to do it for us!

What does a machine need to learn how to solve a task?

1. Data
2. Model family
3. Loss function

## 1.2 Data

Task: Our goal is to predict $Y$ given $X$.

- Ex. For a medical diagnosis, $X$ might record the outcome of a medical test and $Y$ could be whether you have a disease or not. $Y = $ {0, 1}

- Ex. In image classification, $X$ is a picture of an object and $Y$ is the name of the object. $Y = $ {house, boat, car, horse, etc...}

- Ex. For housing price forecasting, $X$ could record the features of your house (size, number of bathrooms/bedrooms, previous price, etc.) and $Y$ is the price tomorrow. $Y = [0, \infty)$

### Collecting data


- Do not know the exact relationship between $X$ and $Y$.
  - We do not know the exact joint distribution of $(X, Y)$
  - or the conditional distribution $Y \mid X$

- Instead, suppose we can sample the distribution of $(X, Y)$
 - that is we have access to instances of $(x_i, y_i)$ drawn from the joint distribution of $(X, Y)$
 - $n$ independent draws $\{(x_1,y_i),...,(x_n,y_n)\}$ from the joint distribution of $(X, Y)$.
 - We call this __training data__

Because we want to learn the relationship between $X$ and $Y$ we need to gather two types of data.

1. __Targets__ $y_1,...,y_n$.
  - Otherwise known as labels or dependent variables.
  - This is our ultimate object of interest. we want to be able to predict future values say $y_{n+1}$

2. __Features__ $x_1,...,x_n$.
  - Otherwise known as covariates or independent variables.
  - This is the information that we use to predict $y$.
  - In statistical learning, the more covariate information the better. Not as interested in selecting (subsetting) variables as in classical statistics but rather quantifying feature importance.
  - We assume there is some relationship between $x_i$ and $y_i$

- We will use the notation $X$ to refer to a random quantity and $x$ to refer to a fixed quantity (data).

- Abstractly we're interested in the relationship between $X$ and $Y$ but we have to settle for the relationship between $x_1,...,x_n$ and $y_1,...,y_n$.
 - And hope the learned relationship __generalizes__ to future data points
 - i.e. that it works for the entire process


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import math

: 

In [ ]:
from scipy.stats import multivariate_normal

x, y = np.mgrid[-3:4:.01, -2:2:.01]
pos = np.dstack((x, y))
ideal = multivariate_normal([0.5, -0.2], [[2.0, 0.8], [0.8, 0.5]])

fig, ax = plt.subplots(1, 2, constrained_layout = True, figsize = (12, 5))
ax[0].contour(x, y, ideal.pdf(pos))
ax[0].set_xlabel('X', fontsize = 20)
ax[0].set_ylabel('Y', fontsize = 20)
ax[0].set_xlim((-4, 5))
ax[0].set_ylim((-3, 3))
ax[0].set_title('Ideal: joint distribution', fontsize = 16)

np.random.seed(0)
realistic = np.random.multivariate_normal([0.5, -0.2], [[2.0, 0.8], [0.8, 0.5]], 500)
ax[1].scatter(realistic[:,0], realistic[:,1])
ax[1].set_xlabel('X', fontsize = 20)
ax[1].set_ylabel('Y', fontsize = 20)
ax[1].set_xlim((-4, 5))
ax[1].set_ylim((-3, 3))
ax[1].set_title('Real: random draws (data)', fontsize = 16)

plt.show()

: 


The process of collecting _good_ data is very challenging.

We will assume that we have access to data already and that it meets the following conditions
1. Each $x_i \in \mathbb{R}^p$ and $y_i \in \mathbb{R}^q$
1. Any pair of observations $(x_i, y_i)$ were collected independently of any $(x_j, y_j)$ for $i \neq j$
2. $(x_i, y_i)$ and $(x_j, y_j)$ have the same distribution for all $i, j$.
3. We have a fixed training sample size $n$, i.e. $1 \leq i \leq n$

Note that all of these assumptions are regularly violated in practice. They just make theory a bit easier.
 - Condition 1 means that all inputs and outputs (features and targets) are real valued vectors (or can be represented as such).
 - Conditions 2 and 3 are collectively know as $i.i.d$ meaning __I__ndependent and __I__dentically __D__istributed.
 - Condition 4 just means we do not plan to keep sampling and updating our model

Unless stated otherwise we will assume our data meets these conditions







Here is another example of data that we will often work with. Pairs of images and labels.
- Each image $x_i$ is a 64 dimesional vector
- Each label $y_i$ is an integer from 0 to 9

In [ ]:
from sklearn import datasets
digits = datasets.load_digits()
x, y = digits.data, digits.target

_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, label in zip(axes, digits.images, digits.target):
    ax.imshow(image, cmap=plt.cm.gray_r)
    ax.set_title("Label: %i" % label)

: 

In [ ]:
image, label

: 

## 1.3 Models and Learning

We now have
1. A task (predict $Y$ given $X$)
2. Some training samples $\{(x_1,y_i),...,(x_n,y_n)\}$

If we knew the joint distribution of $(X, Y)$
- we could find the conditional distribution $Y \mid X$
- this would be the best we could do to describe the relationship between $X$ and $Y$

Instead, we will have to settle for an approximation of the conditional distribution called a __model__ or __predictor__


### A model
In theory a regression model can be any function $f$

$$ f: X \mapsto Y$$


Generally, our model will include an error term to represent our lack of certainty

$$Y = f(X) + \epsilon$$

where $\mathbb{E}[\epsilon] = 0$ and $\text{Var}(\epsilon)= \sigma^2$

Given the the value $X$ we can "predict" $Y$ by simply feeding $X$ through the function $f$

$$\widehat{Y} = f(X)$$

and ignoring the error term.  

__For discussion__: Why is it OK to simply ignore the error term when formulating our prediction?

__Example:__ Linear regression

In linear regression we assume a linear model (prediction function) from $X$ to $Y$ as

$$ Y = \alpha + \beta X + \epsilon$$

and make predictions as

$$\widehat{Y} = \alpha + \beta X$$

after we learn the values of $\alpha$ and $\beta$.

### Model families

As stated in the previous section we have to learn a statistical relationship, which means don't know the statistical relationship, which means we don't know what $f$ is

Instead we can specify the __type of model__ we want to use. Formally, a pre-supposed __family of predictors__ $\mathcal{F}$
  - These are all the possible predictors we will consider. Different problems require different families of predictors.
  - Ex. linear predictors $f \in \mathcal{F}$ s.t. $f(x) = \alpha + \beta x$
  - Impossible to learn an arbitrary predictor from features to targets
  - Instead we impose structure on the relationship between features and targets, such as a linear relationship.
  - $\mathcal{F}$ is also known as a hypothesis class

__Example:__ Linear regression

We suppose $\mathcal{F}$ is all linear models, i.e. models of the form $f(x) = \alpha + \beta x$ for $\alpha, \beta \in \mathbb{R}$. We model $Y$ as

$$ Y = \alpha + \beta X + \epsilon$$

where $\alpha$ and $\beta$ are unknown. Some examples from this class are plotted below

In [ ]:
np.random.seed(1)
x = np.random.normal(1, 1, 200)
y = 0.5 * x + np.random.normal(0, 0.4, 200)

alpha = np.round(np.random.uniform(-1, 1, 5), 2)
beta = np.round(np.random.uniform(-1, 1, 5).reshape(-1, 1), 2)

lm = alpha[:,None] + beta @ x.reshape(1, -1)
lm = lm.T

plt.figure(figsize = (9, 6))
plt.scatter(x, y, alpha = 0.25)
plt.plot(x, lm[:,0], label = f'a = {alpha[0]}, b = {beta[0][0]}')
plt.plot(x, lm[:,1], label = f'a = {alpha[1]}, b = {beta[1][0]}')
plt.plot(x, lm[:,2], label = f'a = {alpha[2]}, b = {beta[2][0]}')
plt.plot(x, lm[:,3], label = f'a = {alpha[3]}, b = {beta[3][0]}')
plt.plot(x, lm[:,4], label = f'a = {alpha[4]}, b = {beta[4][0]}')
plt.xlabel("X", fontsize = 15)
plt.ylabel("Y", fontsize = 15)
plt.title('Random Linear models', fontsize = 15)
plt.legend()
plt.show()

: 

Clearly some of the above lines "fit" the data better than others

How do we determine which line is best? (including those not plotted)

In other words, how do we find the optimal model $f$ from our class $\mathcal{F}$?

### Choosing function classes (model type)

- In general we do not know the appropriate function class, so you will often try many different types of models and see which fits best overall
  - Try linear, sinusoidal, exponential, etc.

- Always assume your function class is __insufficient__
  - i.e., the "true" relationship is not in your supposed class
  - e.g., even when using the class of linear models, the relationship is  almost never truly linear

- Being exactly correct is not the goal
  - Just want to find $f$ with a small loss...
  - ...that continues to perform well on future data


> "All models are wrong, some are useful." -- George Box


: 

: 

: 

The type of model we choose can have an enormous impact on our ability to fit the data.

Remember, in nearly all cases, the model class is __WRONG__.

But clearly, some are more useful than others (i.e. can better approximate the true underlying relationship).

We will discuss ways of choosing models, but ultimately this will come down to experimentation, prior knowledge, and balancing the needs for flexibility, interpretability, and accuracy.

The bulk of this class will focus on evaluating specific model classes and how to choose among different classes.

## 1.4 Loss functions

We now have
1. A task (predict $Y$ given $X$)
2. Some training samples $\{(x_1,y_i),...,(x_n,y_n)\}$
3. A model family $\mathcal{F} =\{ f \mid f:X \mapsto Y \}$

The only outstanding issue is that we dont know which model $f \in \mathcal{F}$ to choose.
- We don't know which $f \in \mathcal{F}$ most accurately represents the relationship between $X$ and $Y$.

To find the optimal function $f$ we need some way to quantitatively measure how good $f$ is. We do this with a __loss function__.

- A loss function is a mathematical function describing how well your model $f$ fits the data $(x_1, y_1),...,(x_n, y_n)$. We generically denote a loss function as
$$ \mathcal{L}(f, (x, y)) $$

- The loss function will return a (usually postive) real number. For example the mean square error loss from regression

$$ \mathcal{L}(f, (x, y)) = MSE(f, (x, y)) =  \frac{1}{n} \sum_{i = 1}^n (y_i - f(x_i))^2 $$

- The closer $f(x_i)$ is to $y_i$ on average, the smaller $MSE(f, (x, y))$ will be.

__Example:__ Least Squares (MSE)

https://www.geogebra.org/m/crBa6TAW

### Optimal models

The loss function is how we define and choose our optimal model. Essentially the optimal model is the model with the smallest loss. Mathematically

$$ \hat{f} = \arg\min_{f \in \mathcal{F}} \mathcal{L}(f, (x, y))$$

That is we choose $\hat f \in \mathcal{F}$ such that
$$
\mathcal{L}(\hat f, (x, y)) \leq \mathcal{L}(f, (x, y)) \quad \forall f \in \mathcal{F}
$$

__Example:__ Linear models with MSE
$$\hat{\alpha}, \hat{\beta} = \arg\min_{\alpha, \beta \in \mathbb{R}} \frac{1}{n} \sum_{i = 1}^n (y_i - \alpha - \beta x_i)^2$$



###Choosing a loss function

- The loss function is how we evaluate models in our model family
- Also defines what it means to be "optimal" (smallest loss)
- Different loss functions lead to different optimal models
   - because the definition of "optimal" changes
- Choosing a loss function is an integral part of model building
- Two example choices (regression)
 - $MSE(f, (x, y)) = \frac{1}{n} \sum_{i =1}^n (y_i - f(x_i))^2$
 - $MAE(f, (x, y)) = \frac{1}{n} \sum_{i =1}^n |y_i - f(x_i)|$

- Properties of your loss function will determine properties of your selected model
  - $MAE$ is more robust to outliers than $MSE$, so the resulting model will not try to fit the outliers as closely as if it was trained by $MSE$
  - In fact, minimizing the $MSE$ fits the mean of the conditional distribution, $E[Y|X]$, while minimizing the $MAE$ fits the conditional median.

: 

In [ ]:
from scipy.optimize import minimize_scalar

## optimize parameters (assume alpha = 0 because it is)


: 

: 

: 

- Using the exact same model type (linear model with no intercept) changing the loss function has clearly changed the predictor.

- In this case, the MAE loss estimated the conditional median while the MSE estimated the conditional mean.

- Choice of loss function can have an even larger impact in more complex models.

- The loss is literally our way of guiding the model fitting process by quantifying how to penalize differences between the estimated and observed values.

### Likelihood-based approaches

In addition to choosing a loss function based on _which properties you want your algorithm to have_, we can choose a natural loss function via the __likelihood__.

> A likelihood is a function specifying how likely particular parameter values are to be the ones that generated the data from this model class.

#### Example
Suppose $X \sim N(\mu, \sigma^2)$.  
- $X$ is the random variable.
- $\theta = (\mu, \sigma)$ are the parameters.
- Likelihood $\mathcal{L}(\theta \mid X=x) = \frac{1}{\sqrt{2\pi \sigma^2}} \exp((x - \mu)^2 / (2 \sigma^2))$

The likelihood function and the density function have the same form.
$$
p(X \mid \theta) = \mathcal{L}(\theta \mid X=x)
$$
However,
- The likelihood is a function of $\theta$ with $X$ considered fixed ($X=x$).
- The density is a function of $X$ with $\theta$ considered fixed.

#### Maximizing likelihoods

The likelihood is a function of $\theta$, which means we can optimize it in terms of $\theta$ (i.e. we can find the value $\hat \theta$ such that $\mathcal{L}(\hat \theta \mid x) \geq \mathcal{L}(\theta \mid x)$ for all $\theta$). More on this later.


Finding $\hat \theta$ in this way is called __maximum likelihood estimation__ (MLE).
- Finds the parameter $\hat \theta$ which was "most likely'' to have produced the data $x$
- _Asymptotic consistency_ - if the data was actually generated with parameter $\theta_0$ then as $n \rightarrow \infty$, $\hat \theta_n \rightarrow \theta_0$.



Extra reading to understand these points better: https://www.math.arizona.edu/~jwatkins/O_mle.pdf

#### MLEs and ML

Suppose we have data sequence $\{(x_i, y_i)\}_{i = 1}^n$ and specify a model of the form
$$
Y = f(X) + \epsilon
$$

where $f \in \mathcal{F}$ and $\epsilon$ is random.

What were actually doing is assuming a distribution on $Y \mid X$

For example
\begin{align}
Y &= f(X) + \epsilon \\
\epsilon &\sim N(0, 1)
\end{align}

Then
$$ Y \mid X = x \sim N(f(x), 1)$$


#### Maximizing the likelihood

The goal in ML is to find the function $f \in \mathcal{F}$ that fits the data as well as possible. We can use maximum likelihood estimation for this.

Suppose we want to find $\hat f = \arg\max_{f \in \mathcal{F}}\mathcal{L}(f \mid \{(x_i, y_i)\}_{i = 1}^n)$ again assuming $Y = f(X) + \epsilon$ with $\epsilon ~ N(0,1)$ where

\begin{align}
\mathcal{L}(f \mid \{(x_i, y_i)\}_{i = 1}^n) &= \prod_{i =1}^n \mathcal{L}(f \mid (x_i, y_i)) \\
&= \prod_{i =1}^n \frac{1}{\sqrt{2\pi}} \exp \left(-\frac{(y_i - f(x_i))^2}{2} \right).
\end{align}

#### Minimization
Almost everything we do will involve minimization rather than maximization (by convention).

You can (and will) show that maximizing the likelihood function is equivalent to minimizing the __negative log likelihood__ function

$$
- \log \prod_{i =1}^n \frac{1}{\sqrt{2\pi}} \exp \left(-\frac{(y_i - f(x_i))^2}{2} \right) \propto \frac{1}{n} \sum_{i =1}^n (y_i - f(x_i))^2
$$

otherwise known as the MSE in this case.


We will see that many other loss functions are derived from maximum likelihood, such as classification losses. You've likely performed MLE many times without realizing it.

## Summary - How do we formulate the learning problem


In a typical problem setup...
- We have two random variables $X$ and $Y$.
- Only $X$ is available for observation.
- Want to predict $Y$ given $X$

Assume some model of the form
$$Y = f(X) + \epsilon$$

Def. A __predictor__ is any "well behaved" function from $X$ to $Y$. Typically call this $f$. We use a predictor to make predictions: $f(X)$

Goal: __learn__ a predictor $f$ such that $f(X) \approx Y$.

The main ingredients to specify a typical learning problem are as follows

1. __Targets__ $y_1,...,y_n$.
  - Otherwise known as labels or dependent variables.
  - This is our ultimate object of interest. we want to be able to predict future values say $y_{n+1}$ given $x_{n+1}$.
2. __Features__ $x_1,...,x_n$.
  - Otherwise known as covariates or independent variables.
  - This is the information that we use to predict $y$. In statistical learning, the more covariate information the better. Not as interested in selecting (subsetting) variables as in classical statistics but rather quantifying feature importance.
  - We assume there is some relationship between $x_i$ and $y_i$
3. The __type of model__ we want to use. Formally, a pre-supposed __family (or class) of predictors__ $\mathcal{F}$
  - These are all the possible predictors we will consider. Different problems require different families of predictors.
  - Ex. linear predictors $f \in \mathcal{F}$ s.t. $f(x) = \alpha + \beta x$
  - Impossible to learn an arbitrary predictor from features to targets
  - Instead we impose structure on the relationship between features and targets, such as a linear relationship.
  - $\mathcal{F}$ is also known as a hypothesis class
4. A __loss__ function $\mathcal{L}(f, (x, y))$
  - Loss functions measure the quality of your predictor. Different loss functions are required for different problems
  - The loss function is how we find the "best" predictor in our pre-supposed family $\mathcal{F}$. I.e. the best model of its kind.
  - Ex. Squared Error $\mathcal{L}(f, (x, y)) = \frac{1}{n} \sum_{i = 1}^n (y_i - f(x_i))^2$


And essentially everything follows through no matter what were doing.
1. Identify the quantitiy (variable) to be predicted
2. Identify information (variables) that might help predict that quantity
3. Choose a model type (linear regression, polynomials, trees, etc.)
4. Find the best model of that type according to your loss function

Or stated another way we will take a "universal" approach to predictive modeling
1. Sample training data $\{(x_i,y_i)\}_{i = 1}^n$
2. Choose a model class $\mathcal{F} = \{f \mid f:X \mapsto Y \}$
3. Find the best model $\hat f = \arg\min_{f \in \mathcal{F}} \mathcal{L}(f, (x, y))$

### Example 1 -- Linear model


Were interested in predicting some target $Y$ using feature $X$. Both are univariate (1 dimension) and plotted against each other.

: 

- There is a clear linear relationship between $X$ and $Y$ values
- Thus we decide to impose a linear model, i.e. we assume
$$ Y = \alpha + \beta X + \epsilon $$
We then further assume that $\alpha = 0$ for simplicity
- Implictly we are defining a family of functions
$$ \mathcal{F} = \{\beta X : \beta \in \mathbb{R} \}$$
- We want to find the "best fitting" member of $\mathcal{F}$ to match our observations

- How to choose? Lets look at some candidates

We sample random functions by sampling random $\beta$ values and plotting $X\beta$

: 

: 

Visually we can see that some of the lines "fit" the data well (blue) and others (purple) are terrible

Lets quantify the fit of a line numerically with a __loss__ function. We will use the __mean squared error__

$$ \mathcal{L}(f, (x, y)) = MSE(f, (x, y)) =  \frac{1}{n} \sum_{i = 1}^n (y - f(x))^2 $$


In [ ]:
# example models have a variety of MSEs


: 

We say that the model with the lowest error is "best"

- How do we find the best possible model? Minimize the loss function.
$$ \hat{f} = \arg\min_{f \in \mathcal{F}} \mathcal{L}(f, (x, y))$$

- Specifically for the MSE
$$ \hat{f} = \arg\min_{f \in \mathcal{F}}  \frac{1}{n} \sum_{i = 1}^n (y_i - f(x_i))^2$$

- More specifically for our problem (we assume $\alpha = 0$)
$$\hat{\beta} = \arg\min_{\beta \in \mathbb{R}} \frac{1}{n} \sum_{i = 1}^n (y_i - \beta x_i)^2$$

- Which has as an exact solution (least squares solution)
$$\hat{\beta} = \frac{\sum_{i = 1}^n x_i y_i}{\sum_{i = 1}^n x_i^2}$$

- So our optimal function is $\hat{f}(x) = \hat{\beta}x$ (plotted below)


In general the loss funtion will not have an exact solution. We will use iterative methods to minimize the loss.



: 

In [ ]:
# optimal model will have the lowest error (average loss)


: 

In [ ]:
# for reference (random guesses)


: 

### Example 2 -- Wave model


Were interested in predicting some target $Y$ using feature $X$. Both are univariate (1 dimension) and plotted against each other.



: 

This time we notice that the relationship between $X$ and $Y$ is more complicated

We suspect there is a __sinusoidal__ relationship so we will restrict ourselves to fitting __sin__ functions


- Thus we decide to impose a sinusoidal model, i.e. we assume
$$ Y = \alpha + \beta \sin(3 \pi X) + \epsilon $$
and assume $\alpha = 0$ for simplicity
- Implictly we are defining a family of functions
$$ \mathcal{F} = \{\beta \sin(3 \pi X) : \alpha \in \mathbb{R}, \beta \in \mathbb{R} \}$$
- We want to find the "best fitting" member of $\mathcal{F}$ to match our observations

- How to choose? Lets look at some candidates

: 

Visually we can see that some of the lines "fit" the data well (blue) and others (purple) are terrible

We again use the __mean squared error__ to quantify the overall "fit" of the candidate models

$$ \mathcal{L}(f, (x, y)) = MSE(f, (x, y)) =  \frac{1}{n} \sum_{i = 1}^n (y_i - f(x_i))^2 $$

Or more specifically

$$ MSE(f, (x, y)) =  \frac{1}{n} \sum_{i = 1}^n (y_i - \beta \sin(3 \pi x_i))^2 $$


In [ ]:


# example models have a variety of errors (average loss)


: 

- Which again has as an exact solution (least squares solution) assuming $\alpha = 0$
$$\hat{\beta} = \frac{\sum_{i = 1}^n \sin(3 \pi x_i) y_i}{\sum_{i = 1}^n \sin(3 \pi x_i)^2}$$

- So our optimal function is $\hat{f}(x) = \hat{\beta}\sin(3 \pi x_i)$ (plotted below)


: 

In [ ]:

# optimal model will have the lowest error (average loss)


: 

In [ ]:
# example models have a variety of errors (average loss)


: 

### Example 1 vs Example 2

- In both cases we colleceted features $x_1,...,x_n$ and targets $y_1,...,y_n$

- In example 1, we plotted the data and suspected a __linear__ relationship so we restricted our search to __linear__ functions of $X$ and $Y$

- In example 2, the relationship was __sinusoidal__ so we restricted our search to __sin__ functions

- Both cases used a mean squared error loss to find the optimal predictor from the pre-defined class of predictors

# 2. Evaluating models

> learning what the machine learns

The central focus of machine learning is teaching our model to make accurate predictions.
- It does not matter how complex your model is
- Or that you understand how exactly it works
- We just care that it continues to _predict well in the future_


The problem we encounter is that machine learning models have become very complex
 - Models like ChatGPT have billions of parameters
 - And are trained on millions of observations

How do we know if our model is going to keep working well?
- In general you don't (anything could change)
- But we can get an idea of when it will and wont fail

There are really three issues that we focus on.

1. **Accuracy** - How accurate is my model? Quantify our ability to predict $Y$ given $X$ on average. We will use the term accuracy to generically refer to our loss
3. **Generalizability** Will my model continue working in the future? If we observe new observation will our model still be accurate and reliable?
4. **Inference** What conclusions can I draw about the relationship between $X$ and $Y$ from my model?



### Part 1 Accuracy

Given target values $y_1,...,y_n$ and predictions $\hat y_1, ..., \hat y_n$, where $\hat y_i = \hat f(x_i)$, **accuracy** genearlly refers to the "similarity" between each $y_i$ and $\hat y_i$.

Examples:
- Regression - suppose each $y_i, \hat y_i \in \mathbb{R}$ for $i \in {1,...,n}$. A common accuracy score for regression problems is the mean squared error (MSE).
$$MSE = \frac{1}{n}\sum_{i = 1}^n (y_i - \hat y_i)^2$$

- Classification - Suppose each $y_i \in {0, 1}$ and each $\hat y_i \in [0, 1]$. A common accuracy score for classification problems is the Binary Cross Entropy (BCE).
$$BCE = -\frac{1}{n}\sum_{i = 1}^n y_i\log(\hat y_i) + (1 - y_i)\log(1 - \hat y_i)$$


Essentially we will measure accuracy with the loss function. Typically lower values of the loss function mean your model is more accurate.

## Part 2 Generalization

Generalization refers to the ability of our model to continue working _in the future_

Suppose
- We learn model $\hat f$ with loss function $\mathcal{L}$ on data sequence $Z_{train} = \{ (x_i, y_i) \}_{i=1}^n$

- The training loss is $\mathcal{L}(\hat f, Z_{train})$ is small enough that we think the model fits the data well

What if we observe another data sequnce?
$Z_{test} = \{ (x_i, y_i) \}_{i={n+1}}^{n + m}$

Will $\mathcal{L}(\hat f, Z_{test})  \approx \mathcal{L}(\hat f, Z_{train})$?

#### Notation

1. We call $Z_{train} = \{ (x_i, y_i) \}_{i=1}^n$ the _training data_ because its the data we use to estimate $f$

2. We call $Z_{test} = \{ (x_i, y_i) \}_{i={n+1}}^{n + m}$ the _testing data_ because its the data we use to test if $f$ will generalize

Our first thought may be to look at the training loss
- We minimized the training loss on a large dataset so that our model would fit the data well
- If it fit this data well shouldn't it fit future data well?

Lets look at an example

We observe the grey datapoints and want to model the relationship between $X$ and $Y$
- We dont see the true relationship (blue line)
- So lets try a few models out

We'll try
1. Linear model
2. A 6th degree polynomial
3. A 12th degree polynomial

: 

: 

On our initial dataset the linear model was __worst__ and the 12th degree polynomial was __best__
- On the data we trained on the most complicated model performed the best
- What about on future data?

Lets suppose we observe some more data
- `x_test` and `y_test`
- How do the three models perform?


We apply each model (trained on the initial dataset) to the new data `x_test` and `y_test` without refitting the model.

: 

In [ ]:
# test error


: 

- Now the order is reversed!?

- Our best model, the 12th degree polynomial is now the worst and linear is the best

- Is this a coincidence?

- Lets check again. We again apply the model (trained on the initial dataset) to new data `x_test` and `y_test` without refitting the model

In [ ]:
### test on test


: 

- Even on this third dataset the linear model is holding strong.

- However
 - Two tests dont prove anything
 - Lets try 200 tests and average the results

: 

: 

: 

Clearly its not enough that our model performs well on the training dataset. Our best performer on training ended up being the worst performer on all of the "future" datasets

### Fitting and Overfitting

Why is this the case? There is phenomenon called __overfitting__ that can (and likely will) happen.

Overfitting means that our learned model is too specialized to the specific dataset that we trained on
- Essentially our model started fitting the "noise" in the data

In this case the true model is a linear model with gaussian white noise around it.
- Our linear model came closest to capturing the underlying "structure" of the model. I.e. the true mean of the conditional distribution.
- the 6th and 12th degree polynomial deviate from the True model because they are trying to fit the individual data points, i.e. noise about the true model


How can we know a situation like this is happening without knowing the true model?

Sample splitting and checking the __generalization gap__
- Split the data into two distinct datasets (train and test)
- Check the difference between training and testing error


## Improving Generalization

Our ultimate goal is to have models that generalize well. There are many strategies that have been developed that can help improve generaliztion.

Some that we will discuss and use

1. Sample splitting
2. Bias / Variance considerations
3. Regularization


Often we will use all three simultaneously

### 1. Sample splitting

The best (and easiest) way to evaluate how well our model performs in the future is to simply evaluate it on some "future" data
- Because its the present we don't have future data
- But we can emulate it through __sample splitting__


A common approach is to randomly split the data into two datasets
 - Train: We use this data to fit (train) our model
 - Test: We use this data to evaluate (test) our model


Essentially all training will be performed on the training dataset and we will reserve the test set for evaluation
 - training is sometimes called "in sample" and test is called "out of sample"


The intuition is that because the model will have never seen the "test" data it is, in no way, fitted to the test data.
- $\hat f$ and $Z_{test} = \{ (x_i, y_i) \}_{i={n+1}}^{n + m}$ are independent of each other
- note: a learned model $\hat f$ is itself a random variable since $\hat f$ is a function of the training data $Z_{train}$ which is random.
- Then $\mathcal{L}(\hat f, Z_{test})$ is a good approximation for performance on unseen future data because it effectively is unseen future data

#### Train, Test, Validation

Another approach is to randomly split the data into three datasets
 - Train, test, and validation
 - Validation acts like a test set when were building our model

You train the model on the training set and check "out of sample" performance on the validation set. You keep repeating this until you're happy with your loss on validation. At the end you use the test set once to verify that the model generalizes.



#### Why is a validation set necessary?

- Lets say you train your model $\hat f$ on $Z_{train} = \{ (x_i, y_i)\}_{i=1}^n$
- $\mathcal{L}(\hat f, Z_{train})$ looks good but $\mathcal{L}(\hat f, Z_{val})$ is too high

- So you go back and modify $f$ and refit on $Z_{train}$

- $\mathcal{L}(\hat f, Z_{train})$ looks good but $\mathcal{L}(\hat f, Z_{val})$ is still too high

- So you go back and modify $f$ ...


Eventually you're happy with $\mathcal{L}(\hat f, Z_{train})$ and $\mathcal{L}(\hat f, Z_{val})$, but there is an issue!

> $\hat f$ is no longer independent of the validation set. You used the validation set (indirectly) to train $\hat f$

$\mathcal{L}(\hat f, Z_{val})$ is no longer an unbiased estimate of the out of sample error. You may very well have overfit to $Z_{train}$ and $Z_{val}$ simultaneously. However, because you've never used $Z_{test}$ you can still use $\mathcal{L}(\hat f, Z_{val})$ to estimate the true out of sample error.

In [ ]:
# classic example of high degree poly failing to generalize?
# from numpy.polynomial.polynomial import Polynomial


: 

- All features of the datset need to be included in the training and testing dataset

: 

- In the original dataset there were `n = 200` observations and `p = 10` features
- Using a 70/30 split
  - train has `n = 140` obs and `p = 10` features
  - train has `n = 60` obs and `p = 10` features

- Always split the rows not the columns (except in special circumstances)


To get train, test, validation just split your first training dataset into a train and test set.

```
           Data
         /      \
    "train"     test
   /       \
Train      Val
```

Most important consider is the number of observations in each set (more is better)
- Typical split may be 50\% Train, 25\% Val, 25\% Test
- Proportions not that important
- raw number of samples more important



#### How does sample splitting help?

The test data is never seen by the model during the fitting process.
 - Performance on test is far more indicative of future performance than performance on train.
 - Because test essentially is "future" data
 - Intuition: If the model generalizes from train to test then its likely that it will generalize again to any future dataset

Drawback: training on less data usually results in a worse model. Need to ensure there is enough data to train an adequate model.

In [ ]:
# classic example of high degree poly failing to generalize?
# from numpy.polynomial.polynomial import Polynomial
np.random.seed(3)

n = 200
beta = 0.5

x = np.linspace(-3, 3, n)
y = beta * x + np.random.normal(0, 3, n)

# 70/30 train/test split
idx = np.random.choice(range(n), int(n*0.7), replace=False)
mask = np.ones(n, dtype=bool)
mask[idx] = False

x_train = x[mask]
y_train = y[mask]
x_test = x[~mask]
y_test = y[~mask]


### fit on train
model1 = np.polyfit(x_train, y_train, deg = 1)
model2 = np.polyfit(x_train, y_train, deg = 6)
model3 = np.polyfit(x_train, y_train, deg = 12)

yhat1_train = np.polyval(model1, np.sort(x_train))
yhat2_train = np.polyval(model2, np.sort(x_train))
yhat3_train = np.polyval(model3, np.sort(x_train))

mse1_train = np.mean((y_train - yhat1_train)**2)
mse2_train = np.mean((y_train - yhat2_train)**2)
mse3_train = np.mean((y_train - yhat3_train)**2)

### evaluate on test
yhat1_test = np.polyval(model1, np.sort(x_test))
yhat2_test = np.polyval(model2, np.sort(x_test))
yhat3_test = np.polyval(model3, np.sort(x_test))

mse1_test = np.mean((y_test - yhat1_test)**2)
mse2_test = np.mean((y_test - yhat2_test)**2)
mse3_test = np.mean((y_test - yhat3_test)**2)

error_df = pd.DataFrame(index = ['Train Error', 'Test Error', 'Gen. Gap'])
error_df['Linear'] = [mse1_train, mse1_test, np.abs(mse1_train - mse1_test)]
error_df['6th Poly'] = [mse2_train, mse2_test, np.abs(mse2_train - mse2_test)]
error_df['12th Poly'] = [mse3_train, mse3_test, np.abs(mse3_train - mse3_test)]

# # generalization gap. Smaller is better. Big differences (usually) mean you way overfit the data
print(error_df)

: 

- A "big" gap between Train Error and Test Error indicates that you (likely) overfit your data
- The word "big" depends on context, so there is no rule
- Between two models with equal training error we prefer the one that has a smaller gap

### 2. Bias and variance trade-off

Assume our general model form $$Y = f(X) + \epsilon$$ where $E[\epsilon] = 0$ and $Var(\epsilon) = \sigma^2$

We learn model $\hat f$ with loss function $\mathcal{L}$ on data sequence $Z_{train} = \{ (x_i, y_i) \}_{i=1}^n$

Given a new data point $x_0$ we decompose the average prediction error

$$
\begin{align}
\text{MSE}(f, (x, y)) &= E[y - \hat y]^2 \\
 &= E[f(x_0) + \epsilon - \hat f(x_0)]^2 \\
 &= E[f(x_0) - \hat f(x_0)]^2 + E[\epsilon]^2 + \underbrace{2E[f(x_0) + \hat f(x_0)]E[\epsilon]}_{E[\epsilon] = 0} \\
 &= E[f(x_0) - \hat f(X)]^2 + E[\epsilon]^2
\end{align}
$$

This last part decomposes as

$$
\underbrace{E[f(x_0) - \hat f(x_0)]^2}_{\text{Reducible error}} + \underbrace{Var[\epsilon]}_{\text{Irreducible error}}
$$

Already we see that our loss function decomposes into two parts
1. Reducible Error - This error between your fitted function and the true underlying. With better models we can hopefully reduce this to 0.
2. Irreducible Error - This is the error intrinsic to our model which assumed there was error in the data

As any self help book will tell you: we shouldn't focus on the things we cant change (irreducible error)

So lets focus on the what we can control: The reducible error


$$
\begin{aligned}
 E[f(x_0) - \hat f(x_0)]^2 &=  E[f - E[\hat f] + E[\hat f] - \hat f)]^2 \\
 &=  E[f - E[\hat f]]^2 + E[E[\hat f] - \hat f]^2 + 2E[f - \hat f]E[E[\hat f] - \hat f]] \\
  &=  E[f - E[\hat f]]^2 + E[E[\hat f] - \hat f]^2 \\
  &=  \underbrace{(f - E[\hat f])^2}_{\text{Bias}(\hat f)^2} + Var[\hat f]
\end{aligned}
$$

So we have decomposed the reducible error into two components. Bring back the $f(x_0)$ notation we have
$$
\text{Reducible Error} = \text{Bias}(\hat f(x_0))^2 + Var[\hat f(x_0)]
$$

- Bias measures the difference between the expected prediction and the underlying true model

- Variance measures how certain we are that were close to the true model
 - i.e. if we trained on a new dataset is our model still likely to be close to the truth?

Explanation:

Recall that the fitted model $\hat f$ is a random variable. Given a random training dataset $Z_{train}$ we learn a random model $\hat f$ via the loss function $\mathcal{L}$


__Bias__ tells us
 - Given a random training dataset $Z_{train}$, how close __on average__ will the learned function $\hat f$ be to the true underlying function $f$
 - i.e. imagine you had 100 datasets $$Z^1_{train},...,Z^{100}_{train}$$
 - you train 100 models $$\hat{f}^1,...,\hat{f}^{100}$$
 - bias $\approx \frac{1}{100} \sum_{i=1}^n [f(x_i) - \hat{f}^i(x_0)]$

__Variance__ tells us
- Given a random training dataset $Z_{train}$, how much will the learned functions $\hat f$ deviate from each other
 - i.e. imagine you had 100 datasets $$Z^1_{train},...,Z^{100}_{train}$$
 - you train 100 models $$\hat{f}^1,...,\hat{f}^{100}$$
 - variance $\approx \frac{1}{n} \sum_{i=1}^n [\hat{f}^i(x_0) - \bar{f}(x_0)]^2$ where $\bar{f}(x_0) = \sum_{i=1}^n \hat{f}^i(x_0)$

Bias measures the how well optimized models from our class $\mathcal{F}$ perform on average across different training sets

Variance measures how similar optimized models are from our class $\mathcal{F}$ on average across different training sets

We want models that are
1. Accurate (low bias)
2. Highly stable (low variance)





In [ ]:
from IPython.display import Image, display
Image('bias_variance_2.png')

: 

Which of these do you think is best?

<div>
<img src="https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/images/bias_variance/bullseye.png" width="500" height="450"/>
</div>

#### Model Complexity and Bias-Variance trade off

The idea of a tradeoff between "bias" and "variance" of our prediction algorithm comes from inspection of the mean squared error loss $MSE$.

- A different loss function will not necesasrily give such a nice decomposition
- What if I use a different loss? Will the principle hold?


Yes

There will always be some tradeoff between your training and testing error


In [ ]:
Image('bias_variance_3.png', width = 700, height = 500)

: 

We can use the idea of __model complexity__ to represent this trade off for any loss function and model family

Statistical Learning Theory tells us
 - More complex models will fit the training data bettter (low bias)
 - Less complex models will generalize better


What is model complexity?
 - We abstractly think of this as the models __capacity__ to fit the data
 - Bigger model families $\mathcal{F}$ allow for more complexity


Theory:

Given a loss function $\mathcal{L}$ and training data $Z_{train} = \{(x_i,y_i)\}_{i=1}^n$, suppose we have two model families $\mathcal{F}_1$ and $\mathcal{F}_2$ such that
$$\mathcal{F}_1 \subset \mathcal{F}_2$$

Example: \\
$\mathcal{F}_1$ is all linear models $\alpha + \beta_1 X$ \\
$\mathcal{F}_2$ is all quadratic models $\alpha + \beta_1 X + \beta_2 X^2$

Any model in $\mathcal{F}_1$ is representable as a model in $\mathcal{F}_2$ by setting $\beta_2 = 0$.

Then
$$\min_{f \in F_2} \mathcal{L}(f, Z_{train}) \leq \min_{f \in F_1} \mathcal{L}(f, Z_{train}) $$


This means that if we have a big family of models that contains a smaller family of models then the big family will always have an equal or better minimizer of the training loss than the small family
- By chosing a bigger family you will never increase the training error
- Up to numerical issues of course


THIS DOES NOT MEAN
$$\min_{f \in F_2} \mathcal{L}(f, Z_{test}) \leq \min_{f \in F_1} \mathcal{L}(f, Z_{test}) $$

<!--
__Example__: \\
$\mathcal{F}_1$ is all linear models $\alpha + \beta_1 X$ \\
$\mathcal{F}_2$ is all cubic models $\alpha + \beta_1 X + \beta_2 X^2 + \beta_3 x^3$ -->

## Conclusion

1. Evaluating on your training dataset is not enough. Otherwise you suffer from overfitting
2. You need to measure out of sample (test) accuracy
   - We do this with sample splitting (train/test) split
3. The loss gives us insight into model behavior
   - Bias and Variance describe accuracy on train and model complexity
   - We want model that are accurate (low bias) and generalize well (low variance)
4. The model complexity gives us insight into performance
 - Balance complexity vs robustness for optimal out of sample predictive skill